# LABORATORIUM 6 - SVD: Zastosowania

Autor: **Dariusz Marecik**

Github link: https://github.com/FloudMe77/SimpleWikiSearch

Celem projektu jest stworzenie wyszukiwarki artykułów wykorzystującej redukcję szumu za pomocą metody SVD.

# Zbiory artykułów

Na potrzeby testów przygotowałem trzy zestawy artykułów, które zostały zapisane w plikach ```.db``` z wykorzystaniem struktury bazy danych SQLite.

## Tworzenie bazy danych na podstawie gotowej paczki artykułów

Projekt rozpocząłem od wykorzystania artykułów z serwisu ```simple.wikipedia.org```. W tym celu pobrałem plik ```.xml``` z pełną zawartością bazy artykułów i poddałem go przetwarzaniu, przygotowując dane do zaimportowania do własnej bazy danych.

Plik ```dump_loader.py```

In [ ]:
import mwxml
import mwparserfromhell
import sqlite3
import re

conn = sqlite3.connect('simplewiki2.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS articles (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                url TEXT,
                title TEXT,
                intro TEXT,
                content TEXT
            )''')
conn.commit()

def clean_text(wiki_text):
    # Parsowanie i usuwanie kodu wiki
    wikicode = mwparserfromhell.parse(wiki_text)
    text = wikicode.strip_code()

    # Usuwanie niechcianych linii (thumb, right, left, center, itp.)
    text = re.sub(r'^.*\b(?:thumb|right|left|center)\b.*$', '', text, flags=re.MULTILINE)

    # Usuwanie nadmiarowych nowych linii, spacji i tabulatorów
    text = re.sub(r'[ \t]+', ' ', re.sub(r'\n{2,}', '\n\n', text))

    # Usuwanie resztek formatowania
    text = re.sub(r'\b(?:[0-9]+\s*px|[0-9]+x[0-9]+px|thumb|right|left|center)\b\|?', '', text, flags=re.IGNORECASE)

    return text.strip()

def count_words(text):
    return len(re.findall(r'\b\w+\b', text))

def main():
    dump_path = "dump/simplewiki-latest-pages-articles.xml"
    cnt = 0
    with open(dump_path, 'r', encoding='utf-8') as f:
        dump = mwxml.Dump.from_file(f)

        # transakcje dla większej wydajności
        c.execute('BEGIN TRANSACTION;')

        for page in dump.pages:
            if page.redirect is not None or page.namespace != 0:
                continue  # pomijamy przekierowania i inne przestrzenie nazw

            for revision in page:
                title = page.title
                wiki_text = revision.text
                if not wiki_text:
                    continue

                clean = clean_text(wiki_text)
                if count_words(clean) < 100:
                    continue  

                if cnt % 1000 == 0:
                    print(f'Przetworzono {cnt} artykułów')

                cnt += 1

                intro = clean.split("\n\n")[0].strip()

                url = f"https://simple.wikipedia.org/wiki/{title.replace(' ', '_')}"

                c.execute('''
                    INSERT INTO articles (url, title, intro, content)
                    VALUES (?, ?, ?, ?)
                ''', (url, title, intro, clean))

                # Commit co 1000 artykułów
                if cnt % 1000 == 0:
                    conn.commit()

        # Zakończenie transakcji po przetworzeniu wszystkich stron
        c.execute('COMMIT;')

    conn.close()

if __name__ == "__main__":
    main()

W wyniku przetwarzania danych powstały dwie bazy: jedna zawierająca artykuły o długości przekraczającej 100 słów (127 tys. rekordów) oraz druga z artykułami mającymi ponad 200 słów (63 tys. rekordów).

## Crawlowanie oraz Scrapowanie Sieci

**Drugim zbiorem jaki chciałem przygotować były artykuły z dziedziny szeroko pojętej historii. W tym celu postanowiłem pobrać artykuły bezpośrednio ze stron wikipedii.**

### Opis działania skryptu scrapującego artykuły historyczne z Wikipedii

Skrypt służy do automatycznego pobierania i zapisywania artykułów związanych z historią z anglojęzycznej Wikipedii. Wykorzystuje techniki web scrapingu oraz filtrację na podstawie słów kluczowych (np. *war*, *empire*, *revolution*, *medieval*), tworząc specjalistyczną bazę danych tekstów historycznych w formacie SQLite (```historywiki.db```).

Działanie programu rozpoczyna się od zadanej strony startowej (w tym przypadku:
```https://en.wikipedia.org/wiki/Category:History_by_location```), a następnie rekurencyjnie przeszukuje powiązane linki do ustalonej głębokości. Każdy artykuł jest analizowany, a jego zawartość oczyszczana z przypisów, znaczników i zbędnych znaków.

Aby umożliwić wznawianie pracy po przerwie, skrypt cyklicznie zapisuje swój stan (kolejkę URL-i, liczniki) do pliku .pkl z użyciem biblioteki pickle. Dodatkowo stosowane są losowe opóźnienia między zapytaniami, by nie przeciążać serwerów Wikipedii.

Do bazy danych jest zapisywana cała treść artykułu, bez usuwania stopwords, czy innych elementów preprocesingu.

#### Schemat działania algorytmu

1. **Start**

   * Ustawienie strony początkowej:
     `https://en.wikipedia.org/wiki/Category:History_by_location`
   * Wczytanie lub inicjalizacja kolejki URL-i i stanu z pliku `.pkl`.

2. **Przeszukiwanie**

   * Rekurencyjne odwiedzanie stron do zadanej głębokości.
   * Filtrowanie linków na podstawie słów kluczowych w tytule i kategoriach.
   * Dodawanie nowych linków do kolejki.

3. **Pobieranie treści**

   * Dla zakwalifikowanych artykułów: tytuł, lead, pełna treść.

4. **Czyszczenie i zapis**

   * Usuwanie przypisów, tagów, nadmiarowych spacji.
   * Zapis do bazy `historywiki.db`.

5. **Zarządzanie stanem i etykieta**

   * Zapisywanie stanu do `.pkl`.
   * Losowe opóźnienia między zapytaniami.

Ostatecznie udało się pobrać ok 90 tys. artykułów

Plik ```scraper.py```

In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
from urllib.parse import quote
import re
import time
from urllib.parse import urlparse, unquote
from collections import deque
import numpy as np
import pickle

KEYWORDS = {
    'history', 'historical', 'historian', 'historiography', 'ancient', 'medieval', 
    'renaissance', 'enlightenment', 'modern', 'prehistoric', 'neolithic', 
    'bronze age', 'iron age', 'classical', 'middle ages', 'dark ages', 
    'colonial', 'victorian', 'industrial', 'postmodern', 'contemporary', 
    'century', 'millennium', 'era', 'period', 'antiquity', 'prehistoric', 
    'pre-columbian', 'post-war', 'interwar', 'war', 'battle', 'siege', 
    'invasion', 'conquest', 'conflict', 'crusade', 'rebellion', 'revolt', 
    'uprising', 'insurrection', 'civil war', 'world war', 'campaign', 
    'revolution', 'insurgency', 'raid', 'guerrilla', 'occupation', 
    'resistance', 'combat', 'skirmish', 'offensive', 'warfare', 'empire', 
    'kingdom', 'dynasty', 'monarchy', 'republic', 'state', 'nation', 
    'civilization', 'colony', 'realm', 'territory', 'commonwealth', 'province', 
    'principality', 'duchy', 'sultanate', 'caliphate', 'confederation', 
    'federation', 'union', 'horde', 'khanate', 'shogunate', 'chiefdom'
}

def save_data(Q, visited, cnt):
    with open("saved_data/scraper3.pkl", "wb") as f:
            pickle.dump({
                "Q": Q,
                "visited": visited,
                "cnt": cnt
            }, f)
def read_data():
    with open("saved_data/scraper3.pkl", "rb") as f:
            data = pickle.load(f)
            Q = data["Q"]
            visited = data["visited"]
            cnt = data["cnt"]
    return Q, visited, cnt

def get_title_from_url(url):
    path = urlparse(url).path 
    title = path.split('/')[-1] 
    return unquote(title.replace('_', ' ')) 

conn = sqlite3.connect('historywiki.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS articles (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                url TEXT,
                title TEXT,
                intro TEXT,
                content TEXT
            )''')
conn.commit()

def clean_wikipedia_text(text):
    # 1. Wyciągamy główną treść artykułu
    match = re.search(
        r"From Wikipedia, the free encyclopedia(.*)",
        text,
        re.DOTALL
    )
    content = match.group(1) if match else text

    # 2. Usuwamy przypisy w nawiasach kwadratowych, np. [1], [b]
    content = re.sub(r"\[\w{1,3}\]", "", content)

    # 3. Usuwamy fragmenty zawierające "vte"
    content = re.sub(r"\bvte\b", "", content)

    # 4. Rozdzielamy camelCase / PascalCase np. "CategoryReferance" → "Category Referance"
    content = re.sub(r"(?<=[a-z])(?=[A-Z])", " ", content)

    # 5. Usuwamy nadmiarowe spacje i taby
    content = re.sub(r"[ \t]{2,}", " ", content)

    # 6. Redukujemy wiele nowych linii do pojedynczego
    content = re.sub(r"\n{2,}", "\n", content)

    # 7. Usuwamy puste linie i whitespace
    content = "\n".join(line.strip() for line in content.splitlines() if line.strip())

    return content

def is_history_related(title):
    return any(keyword in title.lower() for keyword in KEYWORDS)

def get_content(content, content_div, url, title):
    content = clean_wikipedia_text(content)

    # Inicjalizacja intro_paragraphs
    intro_paragraphs = ""
    
    if content_div:
        for elem in content_div.find_all(['p', 'h2']):
            if elem.name == 'p':
                text = elem.get_text(strip=True)
                if text and len(text) > 70:
                    intro_paragraphs = text
                    break
                    
        # Sprawdź czy artykuł już istnieje w bazie danych
        c.execute("SELECT id FROM articles WHERE url = ?", (url,))
        existing = c.fetchone()
        
        if not existing:  # Zapisz tylko jeśli nie istnieje
            c.execute('''
                INSERT INTO articles (url, title, intro, content)
                VALUES (?, ?, ?, ?)
            ''', (url, title, intro_paragraphs, content))
            conn.commit()
            print(f"Zapisano artykuł: {title}")
        else:
            print(f"Artykuł już istnieje w bazie: {title}")

def search(url, maxdepth, read = False):

    Q = deque()
    Q.append((url,0))
    visited = set()
    cnt=0
    last_request_start_time = time.time()
    if read:
        Q,visited,cnt = read_data()
    while Q:
        url,depth = Q.pop()
        
        if depth > maxdepth or url in visited:
            continue
            
        print(f"Przetwarzanie: {url} (głębokość: {depth})")
        visited.add(url)
        current_time = time.time()
        time_since_last_start = current_time - last_request_start_time
        
        desired_interval = np.random.uniform(0.4, 0.6)
        
        sleep_duration = desired_interval - time_since_last_start
        
        if sleep_duration > 0:
            time.sleep(sleep_duration)
            
        # Zapisz czas tuż przed wysłaniem nowego requestu
        last_request_start_time = time.time() 
        
        try:
            headers = {
                'User-Agent': 'own project HistoryWikiSearch/1.0',
                'From': 'marecik7@gmail.com' 
            }

            r = requests.get(url, headers=headers)
            r.raise_for_status()
        except requests.RequestException as e:
            print(f"Błąd pobierania {url}: {e}")
            continue
        cnt+=1
        if cnt%100 ==0:
            save_data(Q, visited, cnt)
        
        
        soup = BeautifulSoup(r.text, 'html.parser')
        content_div = soup.find('div', {'id': 'mw-content-text'})
        
        if not content_div:
            continue
            
        # Sprawdź, czy to jest artykuł o historii i zapisz go
        title = get_title_from_url(url)
        if is_history_related(title) and not url.startswith('https://en.wikipedia.org/wiki/Category:'):
            get_content(soup.get_text(), content_div, url, title)
        
        # Przeszukaj linki na stronie
        else:
            for link in content_div.find_all('a')[::-1]:
                href = link.get('href')
                if href and href.startswith('/wiki/'):
                    rest = href[6:]
                    full_url = 'https://en.wikipedia.org' + href
                    
                    # Jeśli to kategoria, przejdź do niej
                    if rest.startswith('Category:') and any(keyword in rest.lower() for keyword in KEYWORDS):
                        Q.append((full_url,depth + 1))
                        
                    # Jeśli to artykuł związany z historią, dodaj go do kolejki
                    elif is_history_related(rest) and not any(rest.startswith(prefix) for prefix in ['Wikipedia:', 'Special:', 'File:', 'Help:', 'Template:']):
                        if depth < maxdepth:
                            Q.append((full_url,depth + 1))
                        
                    
    return cnt

def main():
    print("Rozpoczynam pobieranie artykułów z Wikipedii")
    
    # Początkowa strona kategorii
    start_url = "https://en.wikipedia.org/wiki/Category:History_by_location"
    
    # Maksymalna głębokość przeszukiwania
    max_depth = 5
    
    # Rozpocznij przeszukiwanie
    found_urls = search(start_url, max_depth)
    
    print(f"Znaleziono {found_urls} artykułów związanych z historią")
    
    conn.close()

if __name__ == "__main__":
    main()

# Przetwarzanie tekstów artykułów

Aby zredukować rozmiar macierzy Bag of Words oraz poprawić jakość wyników wyszukiwania, zaimplementowałem funkcję, która przetwarza tekst w następujący sposób:

* Tokenizuje tekst, dzieląc go na pojedyncze słowa,
* Zamienia wszystkie słowa na małe litery,
* Lematyzuje słowa, sprowadzając je do formy podstawowej (np. "cats" -> "cat"),
* Usuwa słowa, które nie są rzeczywistymi wyrazami (np. cyfry, liczby, znaki specjalne).

Plik ```simplifier.py```

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
try:
    word_tokenize("test")
except LookupError:
    nltk.download('punkt') 
try:
    nltk.pos_tag(["test"])
except LookupError:
    nltk.download('averaged_perceptron_tagger') 
try:
    WordNetLemmatizer().lemmatize("cars")
except LookupError:
    nltk.download('wordnet')


class Simplifier:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        # Mapa tagów POS dla lematyzatora
        self.pos_map = {
            'NN': 'n', 'NNS': 'n', 'NNP': 'n', 'NNPS': 'n', # Noun
            'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', # Verb
            'JJ': 'a', 'JJR': 'a', 'JJS': 'a', # Adjective
            'RB': 'r', 'RBR': 'r', 'RBS': 'r'  # Adverb
        }

    def simplify_words(self, content):
        words = word_tokenize(content)
        
        tagged_words = nltk.pos_tag(words)
        
        simplified_lemmas = []
        for word, tag in tagged_words:
            word_lower = word.lower()
            
            # Filtrowanie stop-wordów i tokenów niealfabetycznych
            if word_lower not in self.stop_words and word.isalpha():
                # Domyślnie użyj 'n' (rzeczownik), jeśli tag nie jest w mapie
                pos_tag_for_lemmatizer = self.pos_map.get(tag[:2], 'n')
                lemma = self.lemmatizer.lemmatize(word_lower, pos=pos_tag_for_lemmatizer)
                
                simplified_lemmas.append(lemma)
                    
        return simplified_lemmas

## Klasa pomocnicza (databse_manager)

Dla polepszenia estetyki kodu stworzyłem klasę pomocniczą do obsługi bazy danych.

Plik ```databse_manager.py```

In [ ]:
import sqlite3

class DatabaseManager:
    def __init__(self,database_name):
        self.database_name = database_name
    
    def get_connection(self):
        return sqlite3.connect(f"{self.database_name}.db")
    
    def get_data(self,items, index_tab):
        with self.get_connection() as conn:
            cursor = conn.cursor()
            placeholders = ','.join(str(i + 1) for i in index_tab)
            query = f"SELECT {items} FROM articles WHERE id IN ({placeholders})"
            return cursor.execute(query).fetchall()


# Główny silnik wyszukiwarki

### Opis:

Program implementuje system wyszukiwania dokumentów oparty na technikach przetwarzania tekstu i redukcji wymiarów (SVD), który pozwala na efektywne wyszukiwanie artykułów na podstawie zapytań. Wykorzystuje macierz Bag of Words (BoW) oraz opcjonalnie dekompozycję SVD do redukcji wymiarów i indeksowanie HNSW w celu szybszego wyszukiwania.

### Kroki, które wykonuje program:

1. **Inicjalizacja:**

   * Sprawdza, czy zapisane dane (macierz BoW oraz struktury słów) istnieją, jeśli tak, to je wczytuje, w przeciwnym razie przygotowuje dane do analizy.

2. **Dodawanie artykułów:**

   * Przetwarza artykuł, rozdzielając tekst na słowa, lematyzując je, a następnie tworzy reprezentację macierzy termów (BoW) dla danego dokumentu.

3. **Tworzenie macierzy BoW:**

   * Buduje rzadką macierz BoW, zawierającą liczbę wystąpień słów w dokumentach.

4. **Obliczanie TF-IDF:**

   * Oblicza ważność słów za pomocą IDF i normalizuje dokumenty, tworząc znormalizowaną macierz TF-IDF.

5. **Obsługa zapytań:**

   * Dla zapytania oblicza podobieństwo z dokumentami za pomocą macierzy BoW lub przestrzeni zredukowanej (SVD), zwracając najbardziej podobne artykuły.

6. **Redukcja wymiarów (SVD):**

   * Jeśli włączona jest opcja SVD, wykonuje dekompozycję macierzy BoW, a następnie używa zredukowanej przestrzeni wektorowej do szybszego wyszukiwania przy użyciu indeksu HNSW.

7. **Indeksowanie HNSW:**

   * Buduje i wykorzystuje indeks HNSW (Hierarchical Navigable Small World) w celu przyspieszenia wyszukiwania podobnych dokumentów w zredukowanej przestrzeni.

8. **Zapis i odczyt danych:**

   * Program zapisuje i wczytuje macierze BoW oraz wyniki dekompozycji SVD do plików w celu późniejszego użycia.

9. **Normalizacja zapytania:**

   * Przed obliczeniem podobieństwa zapytanie jest normalizowane, aby uzyskać spójne wyniki.

Początkowo macierz BOW jest trzymana jako lista krotek, później jest konwertowana do csc_matrix. Obsługiwane jest działanie wyszukiwarki z SVD + HNSW oraz bez tych struktur.

plik ```search_engine.py```

In [ ]:
from collections import defaultdict
from scipy.sparse import csr_matrix, csc_matrix, diags, linalg
import numpy as np
import heapq
from sklearn.preprocessing import normalize
from scipy.sparse import save_npz, load_npz
import pickle
from scipy.sparse.linalg import svds
import simplifier
import hnswlib
import os

class Engine:

    def __init__(self, database_name = '', svd_on = False, k = None):
        # svd_on - determinuje, czy używamy svd przy wyszukiwaniu, czy nie
        # k - liczba największych wartości osobliwych (singular values) w SVD

        word_matrix_path = f"saved_data/csc_BOW_{database_name}.npz"
        word_structures_path = f"saved_data/word_structures_{database_name}.pkl"
        self.is_matrix_saved = self.file_exist(word_matrix_path) and self.file_exist(word_structures_path)
        self.database_name = database_name

        if self.is_matrix_saved:
            self.read_BOW_from_file()
        else:
            print(self.file_exist(word_matrix_path))
            print(self.file_exist(word_structures_path))
            self.number_to_word = []
            self.word_to_number = dict()
            self.tuple_BOW = []
            self.n_articles = 0
            self.csc_BOW = None 
            self.articles_with_word = defaultdict(int)
            self.simplifier = simplifier.Simplifier()
        
        
        self.svd_on = svd_on
        self.k = k

    def file_exist(self,path_name):
        return os.path.exists(path_name) and os.path.isfile(path_name)

    def content_to_tuple_matrix(self, words, id):
        unique_words = set()
        counts = defaultdict(int)
        for word in words:
            if word not in self.word_to_number:
                # następny wolny numerek
                self.word_to_number[word] = len(self.number_to_word)
                self.number_to_word.append(word)

            if word not in unique_words:
                # inkrementacja liczby artykułów z tym słowem
                self.articles_with_word[word] += 1
                unique_words.add(word)
                
            counts[self.word_to_number[word]] += 1
            
        return [(id, col, val) for col, val in counts.items()]

    def add_article(self, id, content):
        words = self.simplifier.simplify_words(content)
        # indeksy w bazie danych zaczynają się od 1, a w macierzy od 0
        new_tuples = self.content_to_tuple_matrix(words, id-1)
        
        self.tuple_BOW.extend(new_tuples)
        self.n_articles += 1

    def create_csr_matrix(self):
        print("start_create_csr_matrix")
        
        if not self.tuple_BOW:
            return csr_matrix((0, 0))
        
        rows, cols, data = zip(*self.tuple_BOW)
        shape = (self.n_articles, max(cols) + 1)
        print("end_create_csr_matrix")
        return csr_matrix((data, (rows, cols)), shape=shape)

    def start_engine(self):
        if not self.is_matrix_saved:
            self.IDF_and_normalization()
            self.save_BOW_to_file()

        if self.svd_on:
            if self.file_exist(f"saved_svd/svd{self.k}_{self.database_name}.pkl"):
                self.read_SVD_from_file()
            else:
                self.lower_rank()

    def IDF_and_normalization(self):
        print("start idf")

        self.csc_BOW = self.create_csr_matrix()  # ustawia self.csc_BOW (TF)

        self.info()

        N = self.csc_BOW.shape[0]  # liczba dokumentów
        M = self.csc_BOW.shape[1]  # liczba słów

        idf = [np.log(N / self.articles_with_word[self.number_to_word[i]]) for i in range(M)]
        self.idf_diag = diags(idf)
        tf_idf = self.csc_BOW @ self.idf_diag

        # Transpozycja: wiersze = słowa, kolumny = dokumenty
        tf_idf = tf_idf.T

        # Normalizacja  dokumentów
        tf_idf = normalize(tf_idf, axis=0, norm='l2')
        self.csc_BOW = tf_idf 
        print("end idf")

    def handleQuery(self, query_vector, top):
        # w zależności czy svd
        return self.handleQueryUVD(query_vector, top) if self.svd_on else self.handleQueryNormal(query_vector, top)
    

    def handleQueryNormal(self, query_vector, top):
        normalized_query = query_vector / linalg.norm(query_vector)
        result = np.abs((normalized_query.T @ self.csc_BOW)).T  # (N, 1)
        similarities = result.flatten()
        top_indices = heapq.nlargest(top, range(len(similarities)), key=lambda i: similarities[i])

        return [(i, round(similarities[i]*100,1)) for i in top_indices]
    
    def lower_rank(self):
        print("start decomposition")
        U, D, Vt = svds(self.csc_BOW, k=self.k)

        self.U = U
        self.Vt = Vt
        self.D = diags(D)
        self.D_values = D.astype('float32')  # przyda się później

        # Przekształcamy dokumenty do przestrzeni zredukowanej
        X_reduced = (np.diag(D) @ Vt).T.astype('float32')  # shape: (n_docs, k)

        # Budujemy HNSW index
        dim = self.k
        self.index = hnswlib.Index(space='cosine', dim=dim)
        self.index.init_index(max_elements=X_reduced.shape[0], ef_construction=200, M=32)
        self.index.add_items(X_reduced)
        self.index.set_ef(200)

        print("end decomposition + HNSW")
        self.save_SVD_to_file()

    def handleQueryUVD(self, query_vector, top=10):
        # if self.idf_diag:
        #     query_vector = self.idf_diag @ query_vector

        norm = linalg.norm(query_vector)
        if norm == 0:
            return []

        normalized_query = query_vector / norm

        q = self.U.T @ normalized_query  
        q = self.D @ q
        q_dense = q.flatten().astype('float32').reshape(1, -1)

        # Szukanie przez HNSW
        labels, distances = self.index.knn_query(q_dense, k=top)

        return [(int(i), round((1 - d) * 100, 1)) for i, d in zip(labels[0], distances[0])]

    def info(self):
        print(self.csc_BOW.shape)
    
    def save_BOW_to_file(self):
        print("start saving BOW")

        save_npz(f"saved_data/csc_BOW_{self.database_name}.npz", self.csc_BOW)
        with open(f"saved_data/word_structures_{self.database_name}.pkl", "wb") as f:
            pickle.dump({
                "number_to_word": self.number_to_word,
                "word_to_number": self.word_to_number,
                "idf_diag": self.idf_diag
            }, f)
        print("end saving BOW")

    def read_BOW_from_file(self):
        print("start reading BOW")

        self.csc_BOW = load_npz(f"saved_data/csc_BOW_{self.database_name}.npz")
        with open(f"saved_data/word_structures_{self.database_name}.pkl", "rb") as f:
            data = pickle.load(f)
            self.number_to_word = data["number_to_word"]
            self.word_to_number = data["word_to_number"]
            if "idf_diag" in data:
                self.idf_diag = data["idf_diag"]
            else:
                self.idf_diag = None
        print("end reading BOW")

    def save_SVD_to_file(self):
        print("start saving SVD")

        with open(f"saved_svd/svd{self.k}_{self.database_name}.pkl", "wb") as f:
            pickle.dump({
                "U": self.U,
                "D": self.D,
                "Vt": self.Vt,
                "index": self.index
            }, f)
        print("end saving SVD")

    def read_SVD_from_file(self):
        print("start reading SVD")
        with open(f"saved_svd/svd{self.k}_{self.database_name}.pkl", "rb") as f:
            data = pickle.load(f)
            self.U = data["U"]
            self.D = data["D"]
            self.Vt = data["Vt"]
            self.index = data["index"]
        print("end reading SVD")

# Zarządzanie silnikiem wyszukiwania

Klasa Search_engine_manager stanowi warstwę pośrednią między użytkownikiem a silnikiem wyszukiwania tekstów opartych na bazie danych artykułów historycznych. Obsługuje zarówno przetwarzanie zapytań użytkownika, jak i inicjalizację oraz ładowanie danych do silnika wyszukiwania.

Główne funkcje
- Inicjalizacja silnika na podstawie bazy danych (SQLite), z opcjonalnym przetwarzaniem SVD (redukcją wymiarowości).

- Przetwarzanie zapytań tekstowych użytkownika – upraszczanie, lematyzacja, konwersja do formy wektorowej.

- Wyszukiwanie podobnych artykułów na podstawie obliczeń podobieństwa kosinusowego.



In [ ]:
import search_engine
import sqlite3
from collections import defaultdict
from scipy.sparse import csc_matrix
import simplifier

class Search_engine_manager:
    def __init__(self, database_name, start = True, svd_on = True, k = 300):
        self.simplifier = simplifier.Simplifier()
        self.database_name = database_name
        self.en = search_engine.Engine(database_name = database_name, svd_on = svd_on, k = k)
        if not start:
            self.press_db_in_engine()
        self.en.start_engine()

    def parse_query(self, query):
        words = self.simplifier.simplify_words(query)
        counts = defaultdict(int)
        for word in words:
            if word in self.en.word_to_number:
                counts[self.en.word_to_number[word]] += 1
        indices, values = zip(*counts.items()) if counts else ([], [])
        n_words = len(self.en.number_to_word)
        return csc_matrix((values, (indices, [0] * len(indices))), shape=(n_words, 1))
    
    def hendle_query(self,query):
        query_vector = self.parse_query(query)
        return self.en.handleQuery(query_vector, 10)

    def press_db_in_engine(self):
        print("start parsing database")
        conn = sqlite3.connect(f"{self.database_name}.db")
        cursor = conn.cursor()
        cursor.execute('SELECT id, content FROM articles ')
        for id,content in cursor.fetchall():
            self.en.add_article(id,content)
        print("Matrix built")



# Fronted wyszukiwarki

## Pliki html

Strona główna

plik ```index.html```

```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SimpleWikiSearch</title>
    <link href="{{ url_for('static', filename='styles/style.css') }}"
          rel="stylesheet" />
</head>
<body>
    <h1>
        <div class="card-logo"> </div> SimpleWikiSearch
    </h1>
    <form action="/flask_app" method="get">
        <input type="text" name="fraze" id="fraze" placeholder="Enter phrase" />
        <button type="Search">Search</button>
    </form>
</body>
</html>

```

Wyniki wyszukiwania

plik ```search_result.html```

```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" intro="width=device-width, initial-scale=1.0">
    <title>Results for {{ title }}</title>
    <link href="{{ url_for('static', filename='styles/style.css') }}" rel="stylesheet" />
</head>
<body>
    <h1>
        <div class="card-logo"> </div> SimpleWikiSearch
        
    </h1>
    
    <form action="/flask_app" method="get">
        <input type="text" name="fraze" id="fraze" placeholder="Enter phrase" />
        <button type="submit">Submit</button>
    </form>

    <hr style="border: none; height: 1px; background-color: black; width: 50em;">
    <h3>Query: {{ title }}</h3>
    <hr style="border: none; height: 1px; background-color: black; width: 50em;">
    <div class="search_result">
        {% for item in results %}
        <div class = "rate">
            
        </div>
        <div class = "result">
            <P1>{{item.rate}}%</P1>
            <h2> <a href="{{ item.url }}">{{item.title}}</a> </h2>
            <p2> {{item.intro}}</p2>
        </div>
        {% endfor %}
    </div>
</body>
</html>

```

Plik css

Jeden wspólny dla każdego pliku html

```
/* Układ ogólny */
.card-logo {
    width: 30px;
    height: 30px;
    background-color: #4285f4;
    border-radius: 50%;
    margin-right: 15px;
}

body {
    background-color: #fff;
    font-family: Arial, sans-serif;
    margin: 20;
    padding: 20px;
    display: flex;
    flex-direction: column;
    align-items: center;
}

/* Nagłówek wyników */
h1 {
    display: flex;
    font-size: 2rem;
    color: #202124;
    margin-bottom: 2em;
    align-items: center;
}
h3 {
    font-size: 1.2rem;
    color: #202124;
    text-align: left; 
    border-bottom: 0;
    border-top: 0;
}

/* Formularz wyszukiwania */
form {
    display: flex;
    flex-direction: row;
    justify-content: center;
    align-items: center;
    width: 100%;
    max-width: 50em;
    margin-bottom: 30px;
}

/* Pole wyszukiwania */
input[type="text"] {
    flex: 1;
    padding: 15px;
    border: 2px solid #eee;
    border-radius: 8px 0 0 8px;
    font-size: 16px;
    outline: none;
}

input[type="text"]:hover {
    box-shadow: 0 1px 4px rgba(121, 123, 128, 0.45);
}

input[type="text"]:focus {
    box-shadow: 0 1px 8px rgba(121, 123, 128, 0.45);
}

/* Przycisk */
button {
    padding: 15px 25px;
    border: 2px solid #eee;
    background-color: #4285f4;
    color: white;
    border: none;
    border-radius: 0 8px 8px 0;
    cursor: pointer;
    font-size: 16px;
}

button:hover {
    background-color: rgb(14, 140, 203);
    box-shadow: 0 1px 8px rgba(121, 123, 128, 0.45);
}

/* Wyniki wyszukiwania */
.search_result {
    width: 100%;
    max-width: 50em;
}

/* Każdy wynik */
.result {
    padding: 20px;
    margin-bottom: 20px;
    border-radius: 12px;
    transition: box-shadow 0.3s ease;
}

.result:hover {
    box-shadow: 0 2px 8px rgba(0,0,0,0.15);
}

/* Tytuł i link */
.result h2 {
    margin: 0 0 10px;
    font-size: 1.2rem;
    color: #1a0dab;
}

.result h2 a {
    text-decoration: none;
    color: inherit;
}

.result p2 {
    display: -webkit-box;
    -webkit-line-clamp: 2;       /* liczba linii */
    -webkit-box-orient: vertical;
    overflow: hidden;
    text-overflow: ellipsis;
    margin: 0;
    color: #4d5156;
    font-size: 14px;
    line-height: 1.5;
    
}
.result p1 {
    color: #b13030;
    font-size: 14px;
    line-height: 1.5;
}

@media (max-width: 300px), (max-height: 300px) {
    body {
        background-color: #000;
    }
}
```

## Serwer

Prosty serwer obsługujący zapytania zadane przez użytkownika

In [ ]:
from flask import Flask, render_template, request
import search_engine_manager
import sqlite3
import database_manager

app = Flask(__name__)

# Configuration
DATABASE_NAME = 'simplewiki100'
SEARCH_ENGINE = search_engine_manager.Search_engine_manager(DATABASE_NAME, svd_on=True, k=300)
DB_MANAGER = database_manager.DatabaseManager(DATABASE_NAME)


def get_data(items, index_tab):
    """Fetch data from the database based on given item fields and indices."""
    conn = sqlite3.connect(f"{DATABASE_NAME}.db")
    cursor = conn.cursor()
    placeholders = ','.join(str(i + 1) for i in index_tab)
    query = f"SELECT {items} FROM articles WHERE id IN ({placeholders})"
    return cursor.execute(query).fetchall()


@app.route('/')
@app.route('/index')
def index():
    """Render the index page."""
    return render_template('index.html')


@app.route('/flask_app')
def get_search():
    """Handle search queries and render search results."""
    fraze = request.args.get("fraze")
    raw_data = SEARCH_ENGINE.hendle_query(fraze)
    indexes, rates = zip(*raw_data)
    indexes = list(indexes)
    rates = list(rates)

    results = [
        {"url": url, "title": title, "intro": intro}
        for url, title, intro in DB_MANAGER.get_data("url, title, intro", indexes)
    ]

    for i, rate in enumerate(rates):
        results[i]["rate"] = rate

    return render_template("search_result.html", title=fraze, results=results)


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


# Testowanie wyszukiwarki

Testy zostały wykonane na bazie artykułów ```simplewiki```, dla artykułów powyżej 100 słów

## Hasło "Joseph Stalin"

| Pozycja | Bez SVD                                      | SVD k=100               | SVD k=200                     | SVD k=300                | SVD k=500                | SVD k=1000                   |
|---------|-----------------------------------------------|-------------------------|-------------------------------|--------------------------|--------------------------|------------------------------|
| 1       | Joseph Stalin (58.7)                          | Vladimir Lenin (80.7)   | Joseph Stalin (80.6)          | Soviet Union (78.1)      | Soviet Union (72.4)      | Joseph Stalin (67.8)         |
| 2       | Joseph Smith (57.4)                           | Leon Trotsky (79.7)     | Mikhail Gorbachev (79.7)      | Joseph Stalin (77.8)     | Joseph Stalin (71.5)     | Nikita Khrushchev (57.6)     |
| 3       | Joseph (name) (45.3)                          | Joseph Stalin (79.7)    | Nikita Khrushchev (78.1)      | Mikhail Gorbachev (75.4) | Nikita Khrushchev (69.5) | Joseph (name) (55.1)         |
| 4       | De-Stalinization (42.7)                       | Nikita Khrushchev (79.0)| Georgy Malenkov (78.0)        | Nikita Khrushchev (74.8) | Yuri Andropov (68.6)     | De-Stalinization (55.1)      |
| 5       | Khrushchev's Secret Speech (35.2)             | Nicholas II (78.8)      | Lavrenty Beria (77.8)         | De-Stalinization (74.8)  | De-Stalinization (68.3)  | Georgy Malenkov (54.3)       |
| 6       | Saint Joseph's Day (33.5)                     | Catherine II (78.8)     | Ryutin Affair (77.7)          | Georgy Malenkov (74.8)   | Georgy Malenkov (68.2)   | Lavrenty Beria (53.5)        |
| 7       | List of people from St. Joseph, Missouri (32.5)| Lavrenty Beria (77.8)   | Hungarian Revolution (77.6)   | Lavrenty Beria (74.5)    | Lavrenty Beria (68.0)    | Khrushchev's Secret Speech (52.8) |
| 8       | Joseph: King of Dreams (32.5)                 | Richard Pipes (77.7)    | Perseus (spy) (77.3)          | Ryutin Affair (74.4)     | Ryutin Affair (67.5)     | Sergei Kirov (52.7)          |
| 9       | Svetlana Alliluyeva (31.2)                    | Perseus (spy) (77.7)    | Mikhail Kalinin (77.2)        | Mikhail Kalinin (73.7)   | Mikhail Kalinin (66.8)   | Akaki Mgeladze (52.6)        |
| 10      | Akaki Mgeladze (31.0)                         | Stepan Bandera (77.7)   | Sixtiers (76.9)               | Sixtiers (73.2)          | Sixtiers (66.6)          | Benjamin (52.6)              |


Dla zapytania „Joseph Stalin” najlepsze rezultaty osiągane są przy zastosowaniu SVD z k = 100–200. Model w tym zakresie trafnie rozpoznaje powiązane postacie i wydarzenia, takie jak Lenin, Trotsky czy Ryutin Affair, oddając historyczny kontekst zapytania.

Bez SVD wyniki opierają się głównie na literalnym dopasowaniu słów („Joseph Smith”, „Joseph (name)”), co prowadzi do niskiej trafności semantycznej.

Przy wyższych wartościach k (300–1000) model zaczyna zbyt szeroko interpretować zapytanie – na czoło wysuwają się hasła ogólne, jak „Soviet Union”, co osłabia precyzję odpowiedzi.

## Hasło "bloody battle in modern times"

| Pozycja | Bez SVD                                                          | SVD k=100                    | SVD k=200                                    | SVD k=300                          | SVD k=500                          | SVD k=1000                              |
| ------- | ---------------------------------------------------------------- | ---------------------------- | -------------------------------------------- | ---------------------------------- | ---------------------------------- | --------------------------------------- |
| 1       | Battle (46.8)                                                    | The Lord of the Rings (79.0) | Battle of Kosovo (76.0)                      | Battle (74.0)                      | Battle (69.2)                      | Battle (66.3)                           |
| 2       | Battle of the Philippine Sea (40.7)                              | Longbow (78.0)               | Battle of Verdun (73.9)                      | Battle of Bosworth Field (73.5)    | Battle of Bosworth Field (69.0)    | Battle of Bosworth Field (64.8)         |
| 3       | Second Battle of El Alamein (40.2)                               | Jack Dempsey (77.6)          | Battle of Finnburg (73.8)                    | Battle of Verdun (72.3)            | Battle droid (68.9)                | Battle droid (64.2)                     |
| 4       | Bloody Sunday (36.4)                                             | Battle of Marathon (77.2)    | Battle of Passchendaele (73.6)               | Second Battle of El Alamein (71.5) | Battle of Tannenberg (1914) (68.8) | Battle of Tannenberg (1914) (63.9)      |
| 5       | Battle of Arras (1917) (35.6)                                    | Benedict Arnold (76.9)       | George S. Patton slapping incidents (72.5)   | Battle of Finnburg (71.5)          | Battle of Verdun (68.2)            | Battle of Verdun (63.5)                 |
| 6       | First Battle of El Alamein (33.9)                                | Siegfried Sassoon (76.9)     | Feigned retreat (72.1)                       | Battle of Passchendaele (70.4)     | Second Battle of El Alamein (66.5) | Battle of the Catalaunian Plains (62.9) |
| 7       | Cristero War (33.3)                                              | Ironclad warship (76.8)      | Battle of Amiens (1918) (71.8)               | Second Battle of Ypres (70.4)      | Battle of Arras (1917) (66.0)      | Second Battle of El Alamein (62.7)      |
| 8       | Warfare in eastern Ukraine during the Russo-Ukrainian War (32.5) | Galley (76.4)                | Battle of Haldighati (71.6)                  | Battle of Amiens (1918) (70.3)     | Battle of Amiens (1918) (65.8)     | Battle of Arras (1917) (61.9)           |
| 9       | Battle of Amiens (1918) (32.0)                                   | Winged hussars (76.3)        | Lalitaditya's invasion of Tokharistan (71.4) | Battle of Haldighati (69.7)        | Battle of Ypres (65.7)             | Battle of Amiens (1918) (61.5)          |
| 10      | Liquid modernity (31.8)                                          | Alexius I (76.3)             | Battle of Bila Tserkva (1626) (71.1)         | Battle of Krusi (69.6)             | Battle of Roncevaux Pass (65.3)    | Battle of Ypres (61.4)                  |


Dla zapytania „bloody battle in modern times” – które jest stosunkowo ogólnym hasłem – wyniki osiągają najlepszą trafność przy zastosowaniu SVD z k = 100–200. W tym zakresie model skutecznie identyfikuje istotne wydarzenia i postacie związane z nowoczesnymi bitwami, takie jak „Battle of Kosovo”, „Battle of Verdun” czy „Battle of Passchendaele”, które miały kluczowe znaczenie w XX wieku. Ponadto pojawiają się postacie związane z historią wojskowości, takie jak „Jack Dempsey” czy „Benedict Arnold”, które także pasują do tematyki nowoczesnych, krwawych bitew.

Bez SVD wyniki są głównie efektem dosłownego dopasowania słów kluczowych. Takie hasła jak „Battle of the Philippine Sea” czy „First Battle of El Alamein” pojawiają się, ale nie są w pełni adekwatne do zapytania, ponieważ dotyczą wcześniejszych okresów, co sprawia, że odpowiedzi stają się mniej trafne w kontekście współczesnych konfliktów zbrojnych.

Wartości k powyżej 200 (300–1000) powodują, że model zaczyna zbyt szeroko interpretować zapytanie. W tych przypadkach pojawiają się wyniki takie jak „Battle droid”, które mogą być zrozumiane w kontekście „bitew”, ale nie odpowiadają bezpośrednio na zapytanie o „nowoczesne bitwy”, co osłabia precyzję odpowiedzi.

# Hasło "crimes against jews"

| Pozycja | Bez SVD                              | SVD k=100                            | SVD k=200                            | SVD k=300                            | SVD k=500                            | SVD k=1000                           |
| ------- | ------------------------------------ | ------------------------------------ | ------------------------------------ | ------------------------------------ | ------------------------------------ | ------------------------------------ |
| 1       | Crime (51.7)                         | The Holocaust (85.4)                 | The Holocaust (78.9)                 | Antisemitism (71.0)                  | Crime (73.1)                         | Crime (70.6)                         |
| 2       | Antisemitism (49.2)                  | Holocaust denial (83.5)              | Antisemitism (77.7)                  | Crimes against humanity (70.9)       | Antisemitism (68.5)                  | Antisemitism (66.6)                  |
| 3       | Homicide (48.0)                      | Babi Yar (83.4)                      | Holocaust denial (76.9)              | Antiziganism (70.8)                  | Hate crime (67.4)                    | Hate crime (66.3)                    |
| 4       | Jew (47.5)                           | Yigal Amir (83.3)                    | Babi Yar (76.8)                      | Holocaust inversion (70.8)           | White-collar crime (67.0)            | White-collar crime (64.8)            |
| 5       | Crimes against humanity (47.0)       | Sayyed Razi Mousavi (82.8)           | Antiziganism (76.8)                  | Iași pogrom (70.6)                   | Crimes against humanity (66.9)       | Crimes against humanity (64.2)       |
| 6       | History of the Jews in Europe (46.0) | Yishai Schlissel (81.8)              | Nazi concentration camp (76.5)       | Secondary antisemitism (70.5)        | Holocaust inversion (66.7)           | Holocaust inversion (63.1)           |
| 7       | Jews for Jesus (42.0)                | Racism in Poland (81.8)              | Odessa massacre (1941) (76.4)        | Holocaust uniqueness debate (70.4)   | Antisemitic stereotypes (66.3)       | Antisemitic stereotypes (62.9)       |
| 8       | Antisemitic stereotypes (41.8)       | Weaponization of antisemitism (81.5) | Iași pogrom (76.1)                   | Weaponization of antisemitism (69.9) | Holocaust uniqueness debate (66.1)   | Secondary antisemitism (61.8)        |
| 9       | Anti-Judaism and antisemitism (41.5) | Historical revisionism (81.4)        | Holocaust uniqueness debate (76.1)   | Historical revisionism (69.9)        | Weaponization of antisemitism (65.9) | Weaponization of antisemitism (61.4) |
| 10      | Rhineland massacres (40.6)           | Kraków pogrom (81.1)                 | Weaponization of antisemitism (76.0) | Naliboki massacre (69.6)             | Naliboki massacre (65.8)             | Rhineland massacres (61.3)           |


Dla zapytania „crimes against Jews” – które jest dosyć ogólnym hasłem, obejmującym zarówno współczesne, jak i historyczne zbrodnie – najlepsze wyniki osiągane są przy zastosowaniu SVD z k = 100–200. W tym zakresie model skutecznie identyfikuje najważniejsze wydarzenia związane z przemocą wobec Żydów, takie jak „The Holocaust” (85.4) czy „Babi Yar” (83.4), które miały kluczowe znaczenie w historii XX wieku. Ponadto pojawiają się istotne pojęcia związane z antysemityzmem, jak „Holocaust denial” (83.5) czy „Weaponization of antisemitism” (81.5), które trafnie pasują do zapytania o zbrodnie przeciwko Żydom.

Bez SVD wyniki są głównie efektem dosłownego dopasowania słów kluczowych. Takie hasła jak „Crime” (51.7) czy „Homicide” (48.0) mogą być powiązane z przemocą, ale nie odnoszą się bezpośrednio do tematyki zbrodni przeciwko Żydom, co sprawia, że odpowiedzi są mniej trafne i bardziej ogólne.

Wartości k powyżej 200 (300–1000) powodują, że model zaczyna zbyt szeroko interpretować zapytanie. W tych przypadkach pojawiają się wyniki takie jak „Crime” (73.1) i „Hate crime” (67.4), które mogą być powiązane z przemocą wobec mniejszości, ale są zbyt ogólne i nie odnoszą się bezpośrednio do zbrodni przeciwko Żydom, co osłabia precyzję odpowiedzi. Ponadto, w wyższych wartościach k pojawiają się terminy takie jak „White-collar crime” (67.0), które wydają się mało związane z zapytaniem.

# Hasło "the most famous polish people in the world"

| Pozycja | Bez SVD (Wartość)                      | SVD k=100 (Wartość)            | SVD k=200 (Wartość)         | SVD k=300 (Wartość)         | SVD k=500 (Wartość)         | SVD k=1000 (Wartość)        |
| ------- | -------------------------------------- | ------------------------------ | --------------------------- | --------------------------- | --------------------------- | --------------------------- |
| 1       | Grzegorz Lato (32.7)                   | Principality of Sealand (70.5) | Poland (67.9)               | Poland (65.9)               | Poland (63.5)               | Poland (57.5)               |
| 2       | Tomasz Frankowski (31.1)               | Kickboxing (69.5)              | Warsaw (66.9)               | Warsaw (65.9)               | Warsaw (60.5)               | Warsaw (56.0)               |
| 3       | Czesław Kiszczak (30.0)                | Universal history (69.3)       | Józef Piłsudski (66.3)      | Euzebiusz Smolarek (64.4)   | Józef Piłsudski (60.2)      | Józef Piłsudski (54.5)      |
| 4       | Zygmunt Bauman (28.8)                  | American kickboxing (69.1)     | Grzegorz Lato (64.8)        | Grzegorz Lato (62.9)        | Grzegorz Lato (60.1)        | Grzegorz Lato (54.2)        |
| 5       | Jan Kobuszewski (28.7)                 | Shamanism (68.4)               | Józef Kowalski (64.3)       | Józef Kowalski (62.7)       | Józef Kowalski (60.1)       | Józef Kowalski (54.1)       |
| 6       | Zbigniew Ścibor-Rylski (28.1)          | Four Freedoms (67.0)           | Jan Parandowski (63.4)      | Jan Parandowski (62.5)      | Jan Parandowski (59.5)      | Jan Parandowski (53.2)      |
| 7       | Andrzej Strzelecki (28.0)              | Dariusz Michalczewski (66.3)   | Arkadiusz Milik (63.0)      | Józef Czapski (62.3)        | Józef Czapski (59.0)        | Józef Czapski (52.6)        |
| 8       | Adam Zagajewski (27.7)                 | Johnny Ekström (66.1)          | Wilm Hosenfeld (62.6)       | Arkadiusz Milik (62.3)      | Arkadiusz Milik (58.9)      | Arkadiusz Milik (52.6)      |
| 9       | Polish People's Army (27.4)            | Charlotte Walker (65.9)        | Reuven Fahn (62.6)          | Wilm Hosenfeld (62.0)       | Wilm Hosenfeld (58.9)       | Wilm Hosenfeld (52.4)       |
| 10      | Polish Armed Forces in the East (27.3) | Ekkehard Knobelspies (65.8)    | Włodzimierz Smolarek (62.5) | Włodzimierz Smolarek (61.6) | Włodzimierz Smolarek (58.7) | Włodzimierz Smolarek (52.4) |


Przy k=100: Wyniki są bardzo nietrafione - na pierwszych miejscach znajdują się pozycje jak "Principality of Sealand" (mikronacja), "Kickboxing" czy "Universal history", które nie mają bezpośredniego związku z zapytaniem o znanych Polaków.

Przy k=200-300: Następuje znaczna poprawa trafności. "Poland" pojawia się na pierwszym miejscu, a w top 10 znajduje się wielu znanych Polaków (Józef Piłsudski, Grzegorz Lato, Euzebiusz Smolarek).

Przy k=500-1000: Ranking stabilizuje się, choć wartości liczbowe (w nawiasach) stopniowo maleją wraz ze wzrostem k.
Trend wartości liczbowych: Ogólnie widać, że wartości liczbowe maleją wraz ze wzrostem parametru k. Przy k=100 najwyższa wartość to 70.5, a przy k=1000 spada do 57.5.

SVD skutecznie eliminuje "zapychacze" przy wyższych wartościach k (od 200 wzwyż).
Porównując wyniki bez SVD z wynikami dla większych wartości k (300-1000), widać jak metoda SVD może przekształcić ranking z listy zawierającej mniej znane osoby (np. Jan Kobuszewski, Andrzej Strzelecki) na listę bardziej reprezentatywną dla zapytania o "najsłynniejszych Polaków".

## Podsumowanie testów

- Niskie wartości k zapewniają precyzyjne i trafne wyniki, dobrze odpowiadające zapytaniom.

- Zbyt wysokie wartości k mogą prowadzić do rozmycia wyników, zwiększając liczbę wyników mniej trafnych, przez co dokładność odpowiedzi spada.

- SVD z optymalnym k (100–200) pomaga osiągnąć najlepsze dopasowanie semantyczne do zapytania, skutecznie identyfikując kluczowe tematy i elementy zarówno dla zapytań bardzo ogólnych ("bloody battle in modern times") jak i bardziej konkretnych ("Joseph Stalin")

- Czas wykonania zapytania rośnie wraz ze wzrostem k

# Testy dla zbioru artykułów historycznych z "normalnej" wikipedii

Artykuły pozyskane przez crawler są dużo dłuższe niż te będące na simplewiki, dlatego wyniki dopasowania mogą być inne niż dla simplewiki

## Hasło "fall of Poland"

| **Pozycja** | **Bez SVD (Wartość)**                           | **SVD k=100 (Wartość)**                     | **SVD k=200 (Wartość)**                               | **SVD k=300 (Wartość)**                               | **SVD k=500 (Wartość)**                   |
| ----------- | ----------------------------------------------- | ------------------------------------------- | ----------------------------------------------------- | ----------------------------------------------------- | ----------------------------------------- |
| 1           | Poland–United States relations (43.1)           | History of Poland (94.7)                    | History of Poland (94.0)                              | History of Poland (93.7)                              | History of Poland (93.1)                  |
| 2           | Poland and the European Union (43.0)            | Second Polish Republic (94.7)               | Second Polish Republic (93.9)                         | Second Polish Republic (93.7)                         | Second Polish Republic (93.1)             |
| 3           | Poland–United Kingdom relations (38.9)          | Polish–Czechoslovak border conflicts (94.6) | Military history of Poland during World War II (93.0) | Military history of Poland during World War II (92.1) | History of Poland (1795–1918) (90.7)      |
| 4           | History of Poland (1795–1918) (37.5)            | Polish–Czechoslovak border conflicts (94.6) | History of Poland (1918–1939) (93.0)                  | History of Poland (1918–1939) (92.1)                  | History of Poland (1918–1939) (89.6)      |
| 5           | History of Poland (1918–1939) (37.1)            | Polish–Czechoslovak border conflicts (94.6) | Republic of Poland (93.0)                             | Republic of Poland (92.1)                             | Republic of Poland (89.3)                 |
| 6           | Republic of Poland (36.0)                       | Republic of Poland (94.3)                   | Second Republic of Poland (92.3)                      | Second Republic of Poland (91.8)                      | Second Republic of Poland (89.3)          |
| 7           | Greater Poland Uprising (disambiguation) (35.8) | Second Republic of Poland (93.8)            | Interwar Poland (92.1)                                | Interwar Poland (91.3)                                | Interwar Poland (89.3)                    |
| 8           | Template talk\:Polish uprisings (32.6)          | Interwar Poland (93.8)                      | Third Polish Republic (92.1)                          | Third Polish Republic (91.1)                          | Third Polish Republic (88.3)              |
| 9           | Third Polish Republic (32.4)                    | Polish occupation zone in Germany (93.8)    | Polish contribution to World War II (92.1)            | Polish contribution to World War II (91.1)            | History of Poland under partitions (87.6) |
| 10          | History of Poland under partitions (32.3)       | Republic of Poland (94.3)                   | History of Kraków (91.7)                              | History of Kraków (90.6)                              | History of Kraków (87.5)                  |


Bez SVD: Wyniki koncentrują się na relacjach międzynarodowych Polski (z USA, UE, Wielką Brytanią) zamiast na wydarzeniach historycznych związanych z upadkiem państwa.

Przy k=100: Nastąpiła radykalna zmiana tematyczna - dominują artykuły o historii Polski i II RP. Występują duplikaty (np. "Polish–Czechoslovak border conflicts" na 3 pozycjach) i powtórzenia ("Republic of Poland").

Przy k=200-300: Ranking się stabilizuje i oferuje większą różnorodność historyczną, obejmując II RP, II wojnę światową i III RP. Eliminuje problematyczne duplikaty z k=100.

Przy k=500: Podobne wyniki jak przy k=200-300, ale z nieznacznie niższymi wartościami (87-93). Pojawia się "History of Poland under partitions" bezpośrednio związane z tematyką rozbiorów.

Widzimy generalnie pogorszenie się jakości wyszukiwań. Wynika to z dużo większej liczby danych i też szumu, który nie został w pełni wyeliminowany.

Warto wspomnieć, że Bag Of Words dla ```historywiki``` miał wielkość 1.250.000 co potwierdza duże zaszumienie.

Powtarzające się artykuły wynikają ze specyfiki crowlera. Artykuły są identygikowane przez adres url, ale niestety do niektórych artykułów odsyła więcej niż jeden link.

## Hasło "Relations between Russia and Ukraine"

| Pozycja | Bez SVD (Wartość)                                                                    | SVD k=100 (Wartość)                                                                                 | SVD k=200 (Wartość)                                                                                 | SVD k=300 (Wartość)                                                                  | SVD k=500 (Wartość)                                                                  |
| ------- | ------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------ | ------------------------------------------------------------------------------------ |
| 1       | February 2025 United States–Russia summit in Saudi Arabia (55.0)                     | Diplomatic expulsions during the Russo-Ukrainian War (96.6)                                         | Foreign relations of Russia since the Russian invasion of Ukraine (94.6)                            | Non-government reactions to the Russian invasion of Ukraine (93.5)                   | Non-government reactions to the Russian invasion of Ukraine (92.2)                   |
| 2       | Non-government reactions to the Russian invasion of Ukraine (54.9)                   | Foreign relations of Russia since the Russian invasion of Ukraine (96.6)                            | Government and intergovernmental reactions to the Russian invasion of Ukraine (94.6)                | Foreign relations of Russia since the Russian invasion of Ukraine (93.5)             | Foreign relations of Russia since the Russian invasion of Ukraine (92.2)             |
| 3       | Foreign relations of Russia since the Russian invasion of Ukraine (54.9)             | Government and intergovernmental reactions to the Russian invasion of Ukraine (95.8)                | Reactions to the Russian invasion of Ukraine (94.0)                                                 | Government and intergovernmental reactions to the Russian invasion of Ukraine (92.7) | Government and intergovernmental reactions to the Russian invasion of Ukraine (91.0) |
| 4       | Government and intergovernmental reactions to the Russian invasion of Ukraine (52.0) | Reactions to the Russian invasion of Ukraine (95.2)                                                 | Prelude to the Russian invasion of Ukraine (93.5)                                                   | Reactions to the Russian invasion of Ukraine (92.0)                                  | Reactions to the Russian invasion of Ukraine (90.4)                                  |
| 5       | Reactions to the Russian invasion of Ukraine (50.8)                                  | Prelude to the Russian invasion of Ukraine (94.9)                                                   | International recognition of the Donetsk People's Republic and the Luhansk People's Republic (93.0) | Prelude to the Russian invasion of Ukraine (91.9)                                    | Prelude to the Russian invasion of Ukraine (90.1)                                    |
| 6       | Legality of the Russian invasion of Ukraine (50.5)                                   | Controversy in Russia regarding the legitimacy of eastward NATO expansion (94.8)                    | International reactions to the annexation of Crimea by the Russian Federation (92.9)                | International reactions to the annexation of Crimea by the Russian Federation (91.3) | On conducting a special military operation (89.4)                                    |
| 7       | Foreign Relations of Russia since the Russian invasion of Ukraine (50.1)             | International recognition of the Donetsk People's Republic and the Luhansk People's Republic (94.5) | Prelude to the Russian invasion of Ukraine (92.4)                                                   | Prelude to the Russian invasion of Ukraine (91.3)                                    | Prelude to the Russian invasion of Ukraine (88.6)                                    |
| 8       | Special relationship (international relations) (49.5)                                | International reactions to the annexation of Crimea by the Russian Federation (94.2)                | Foreign Relations of Russia since the Russian invasion of Ukraine (92.4)                            | Foreign Relations of Russia since the Russian invasion of Ukraine (91.1)             | Foreign Relations of Russia since the Russian invasion of Ukraine (88.6)             |
| 9       | List of battles involving the Russian Federation (48.4)                              | Foreign Relations of Russia since the Russian invasion of Ukraine (94.2)                            | Prelude to the 2022 Russian invasion of Ukraine (92.4)                                              | Prelude to the 2022 Russian invasion of Ukraine (91.1)                               | Prelude to the 2022 Russian invasion of Ukraine (88.6)                               |
| 10      | Template talk\:Ukraine–European Union relations (47.3)                               | International reactions to the war in Donbas (94.1)                                                 | International reactions to the war in Donbas (92.4)                                                 | International reactions to the war in Donbas (91.1)                                  | International reactions to the war in Donbas (88.6)                                  |


Bez SVD: Zdominowane przez aktualne wydarzenia polityczne - szczyt USA-Rosja (2025) na pierwszym miejscu, reakcje na inwazję i kwestie prawne konfliktu. Widoczne są też ogólne hasła jak "Special relationship" i techniczne wpisy. Brak historycznej perspektywy stosunków dwustronnych.

Przy k=100: Znacząca poprawa trafności - koncentracja na dyplomatycznych i międzynarodowych aspektach konfliktu. Pojawia się kontekst historyczny (NATO, Donbas, Krym). Uwzględnione są także ważne punkty zwrotne w relacjach (uznanie republik separatystycznych).

Przy k=200: Stabilizacja wyników z naciskiem na relacje międzynarodowe Rosji po inwazji. Więcej kontekstu historycznego (preludium do inwazji) i kluczowych momentów (aneksja Krymu). Widoczne dublowanie niektórych artykułów.

Przy k=300: Podobna struktura do k=200, ale z lepszą spójnością bez duplikatów. "Non-government reactions" awansuje na pierwsze miejsce.

Przy k=500: Pojawia się rosyjska perspektywa ("special military operation"). Większy spadek wartości dla niższych pozycji rankingu. Struktura tematyczna pozostaje skupiona na konflikcie.

 SVD istotnie poprawia trafność wyników, ale nawet optymalne wartości k=200-300 nie rozwiązują problemu jednostronnego skupienia na konflikcie zbrojnym z pominięciem szerszego kontekstu historycznych stosunków rosyjsko-ukraińskich.

# Podsumowanie

### Wnioski:
- SVD znacząco poprawia trafność wyników, szczególnie dla bardziej ogólnych zapytań.
- Optymalne wartości parametru k (100–300) zapewniają najlepszy balans między precyzją a różnorodnością wyników.
- Wyszukiwarka działa skutecznie zarówno na małych, jak i dużych zbiorach danych, choć większe zbiory wymagają bardziej zaawansowanego przetwarzania w celu redukcji szumu.
- Projekt demonstruje potencjał technik przetwarzania języka naturalnego (NLP) w budowie wyszukiwarek semantycznych.